In [1]:
from typing import List

from pathlib import Path
import pandas

from activityMatch import Matching
from loader import load_activities, load_players

In [2]:
activities = load_activities(Path('data/format_standard_activites_cleaned.csv'))
players = load_players(Path('data/format_standard_inscriptions_cleaned.csv'), activities)

Detected 10 columns containing wishes


In [3]:
matching = Matching(players, activities)
matching.print_players_status()
print("")
matching.print_activities_status()

Activities given to each player:
* A | Got 0 activities
* B | Got 0 activities
* C | Got 0 activities
* D | Got 0 activities
* E | Got 0 activities
* F | Got 0 activities
* G | Got 0 activities
* H | Got 0 activities
* I | Got 0 activities
* J | Got 0 activities

Activities with a full cast:
Activities WITHOUT a full cast:
* 0 | ☾ Éclipse ☉ | 0 / 4 players | 2022-08-16 10:00:00 - 2022-08-16 14:00:00
  - 
  - 
  - 
  - 

* 1 | À travers les ténèbres | 0 / 5 players | 2022-08-16 11:00:00 - 2022-08-16 15:00:00
  - 
  - 
  - 
  - 
  - 

* 2 | Au dernier repos | 0 / 6 players | 2022-08-16 12:00:00 - 2022-08-16 16:00:00
  - 
  - 
  - 
  - 
  - 
  - 

* 3 | En suivant l'Alouette | 0 / 7 players | 2022-08-16 13:00:00 - 2022-08-16 17:00:00
  - 
  - 
  - 
  - 
  - 
  - 
  - 

* 4 | Euterpe | 0 / 8 players | 2022-08-16 14:00:00 - 2022-08-16 18:00:00
  - 
  - 
  - 
  - 
  - 
  - 
  - 
  - 

* 5 | Hotel California | 0 / 9 players | 2022-08-16 15:00:00 - 2022-08-16 19:00:00
  - 
  - 
  - 
  - 
  - 


In [4]:
matching.solve()

Casting in priority the players with only one wish and no casts yet
No more priority players. Now casting like usual
Giving [☾ Éclipse ☉] to E
Giving [☾ Éclipse ☉] to A
Giving [Au dernier repos] to F
Giving [Le dortoir des papillons] to G
Giving [Les Enfants de la Nuit d'Eté] to I
Giving [Lumis] to H
Giving [Nostalgie d'innocence] to C
Giving [Que vienne la nuit] to B
Giving [Songes d'une nuit d'hiver] to D
Giving [WINTERHORN] to J
Casting in priority the players with only one wish and no casts yet
No more priority players. Now casting like usual
Giving [À travers les ténèbres] to I
Giving [À travers les ténèbres] to D
Giving [Hotel California] to H
Giving [La nouvelle maison des Barbapapas] to B
Giving [La nouvelle maison des Barbapapas] to J
Giving [Papers] to E
Giving [Pensine] to G
Giving [Que vienne la nuit] to A
Giving [Silence au Quatrième Étage] to F
Giving [WINTERHORN] to C
Casting in priority the players with only one wish and no casts yet
No more priority players. Now castin

In [5]:
matching.print_players_status()

Activities given to each player:
* I | Got 2 activities
  - Les Enfants de la Nuit d'Eté | Start: 2022-08-16 19:00:00
  - À travers les ténèbres | Start: 2022-08-16 11:00:00
* C | Got 2 activities
  - Nostalgie d'innocence | Start: 2022-08-16 21:00:00
  - WINTERHORN | Start: 2022-08-17 05:00:00
* E | Got 3 activities
  - ☾ Éclipse ☉ | Start: 2022-08-16 10:00:00
  - Papers | Start: 2022-08-16 22:00:00
  - Hotel California | Start: 2022-08-16 15:00:00
* D | Got 3 activities
  - Songes d'une nuit d'hiver | Start: 2022-08-17 02:00:00
  - À travers les ténèbres | Start: 2022-08-16 11:00:00
  - Le Dilemme de Chronos | Start: 2022-08-16 17:00:00
* B | Got 3 activities
  - Que vienne la nuit | Start: 2022-08-17 00:00:00
  - La nouvelle maison des Barbapapas | Start: 2022-08-16 16:00:00
  - This is Halloween | Start: 2022-08-17 04:00:00
* G | Got 3 activities
  - Le dortoir des papillons | Start: 2022-08-16 18:00:00
  - Pensine | Start: 2022-08-16 23:00:00
  - Zero Escape Radical Outcome | Star

In [86]:
import pandas

def charger_conflits():
    creneaux = pandas.read_csv('créneaux_interludes', delimiter=',', quotechar='"', parse_dates=[1])
    creneaux = creneaux[creneaux['Ouverte aux inscriptions'] == True]
    creneaux['Durée'] = pandas.to_timedelta(creneaux['Durée'].fillna(creneaux['Durée activité']))
    creneaux['Fin'] = creneaux['Début'] + creneaux['Durée']
    creneaux['Id'] = creneaux.index + 1

    creneaux.drop(['Salle', 'Ouverte aux inscriptions', 'Affiché sur le planning', "Affiché sur l'activité", "Couleur", 'Durée activité', 'Durée'], axis=1, inplace=True)

    conflits = dict((i, []) for i in creneaux['Id'])
    for _, cre in creneaux.iterrows():
        id1, start, end = cre['Id'], cre['Début'], cre['Fin']
        for _, cre2 in creneaux.iterrows():
            if cre['Titre'] == cre2['Titre']:
                continue
            id2, start2, end2 = cre2['Id'], cre2['Début'], cre2['Fin']
        
            if start <= start2:
                if end > start2:
                    conflits[id1].append(id2)
                else:
                    continue
            elif start < end2:
                conflits[id1].append(id2)
            else:
                continue
    
    return conflits

In [107]:
activites = pandas.read_csv('activites_interludes', delimiter=',', quotechar='"')
activites = activites[activites['must_subscribe'] == True].sort_values('title')
for _, a in activites.iterrows():
    print(f"{a['title']}: {a['min_participants']} - {a['max_participants']}")


[GN] Abyme Secret: 6 - 6
[GN] Black Pram: 4 - 4
[GN] Chaos Grown-ups: 6 - 7
[GN] Documents Secrets: 7 - 7
[GN] L'Ecosysteme: 15 - 20
[GN] L'engrenage infernal: 40 - 40
[GN] La première loi: 12 - 12
[GN] Le Choix de Préséphone: 13 - 13
[GN] Le Dilemme de Chronos: 4 - 4
[GN] Lockdown en entreprise: 8 - 8
[GN] Professeur Layton et le mystère de l'île de Soay: 10 - 10
[GN] Sous l'océan: 8 - 8
[GN] Zero Escape : Radical Outcome: 9 - 9
[JV] Abyss Crew: 4 - 12
[JdR] Demi-Dieu: 3 - 5
[JdR] Larmes & Poussières: 4 - 4
[JdR] Pour une poignée d'astro-dollars: 4 - 4
[JdR] Préservez la Flamme: 4 - 5
[JdR] [Pokemon Beta] Le cristal poni: 3 - 5
[JdR] [d100 béta] Les larmes de Melvumia: 2 - 4
[Jeux] Chaîne Alimentaire: 13 - 13
[Jeux] Fresque du Climat: 3 - 16
[Jeux] Tournoi de Belote: 0 - 0


In [110]:
participants, choix, activites, capacites = charger_activites()

Nombre de places max pour [GN] L'engrenage infernal 40
Nombre de places min pour [GN] L'engrenage infernal 35
Nombre de places max pour [GN] Le Dilemme de Chronos 4
Nombre de places max pour [JV] Abyss Crew 12
Nombre de places max pour [GN] Chaos Grown-ups 7
Nombre de places max pour [GN] Professeur Layton et le mystère de l'île de Soay 10
Nombre de places max pour [JdR] Demi-Dieu - Samedi 5
Nombre de places max pour [GN] Lockdown en entreprise 8
Nombre de places max pour [GN] Le Choix de Préséphone 13
Nombre de places max pour [GN] Documents Secrets 7
Nombre de places max pour [JdR] Larmes & Poussières 4
Nombre de places max pour [GN] L'Ecosysteme 20
Nombre de places max pour [GN] La première loi 12
Nombre de places max pour [Jeux] Chaîne Alimentaire 13
Nombre de places max pour [JdR] [Pokemon Beta] Le cristal poni 5
Nombre de places max pour [JdR] Pour une poignée d'astro-dollars 4
Nombre de places max pour [GN] Sous l'océan 8
Nombre de places max pour [JdR] [d100 béta] Les larmes de

In [166]:
participants, choix, activites, capacites = charger_activites()

Nombre de places max pour [GN] L'engrenage infernal 40
Nombre de places min pour [GN] L'engrenage infernal 35
Nombre de places max pour [GN] Le Dilemme de Chronos 4
Nombre de places max pour [JV] Abyss Crew 12
Nombre de places max pour [GN] Chaos Grown-ups 7
Nombre de places max pour [GN] Professeur Layton et le mystère de l'île de Soay 10
Nombre de places max pour [JdR] Demi-Dieu - Samedi 5
Nombre de places max pour [GN] Lockdown en entreprise 8
Nombre de places max pour [GN] Le Choix de Préséphone 13
Nombre de places max pour [GN] Documents Secrets 7
Nombre de places max pour [JdR] Larmes & Poussières 4
Nombre de places max pour [GN] L'Ecosysteme 20
Nombre de places max pour [GN] La première loi 12
Nombre de places max pour [Jeux] Chaîne Alimentaire 13
Nombre de places max pour [JdR] [Pokemon Beta] Le cristal poni 5
Nombre de places max pour [JdR] Pour une poignée d'astro-dollars 4
Nombre de places max pour [GN] Sous l'océan 8
Nombre de places max pour [JdR] [d100 béta] Les larmes de

In [167]:
conflits = charger_conflits()

In [168]:
from copy import deepcopy
# SAVE
choix_bak = deepcopy(choix)
capa_bak = deepcopy(capacites)
conf_bak = deepcopy(conflits)

In [214]:
# LOAD
choix, capacites, conflits = deepcopy(choix_bak), deepcopy(capa_bak), deepcopy(conf_bak)

In [215]:
matchings = match(choix, capacites, conflits)

{27: 40, 5: 4, 37: 12, 18: 7, 21: 10, 8: 5, 11: 8, 15: 13, 19: 7, 16: 4, 9: 20, 26: 12, 17: 13, 14: 5, 24: 4, 10: 8, 2: 4, 3: 6, 6: 4, 20: 5, 13: 9, 28: 16, 31: 5, 23: 16, 29: 16}
First wave to player with a single choice
Found some with only one remaining choice
Second stage with other choices
New wave of assignments
New wave of assignments


In [222]:
named_matchings = dict()
for (act_id, pl) in matchings.items():
    act = activites[act_id]
    n_p = []
    for p in pl:
        n_p.append(participants[p])
    named_matchings[act] = n_p
    
for (act, pl) in named_matchings.items():
    print(f"{act} : {len(pl)} personnes")
    for p in pl:
        print(f" - {p}")

[GN] L'engrenage infernal : 29 personnes
 - Simon Fernandez (Lyon)
 - Adrien Berthelot (Lyon)
 - Nattes Kastylevsky (Lyon)
 - Rémi Tain (Lyon)
 - Antoine Guilmin-Crépon (Paris Saclay)
 - Thomas Lambert (Lyon)
 - Thomas Vitry (Rennes)
 - Marlysa Vivier (Lyon)
 - Maxime Cauté (Rennes)
 - Constantin Vaillant-Tenzer (Ulm)
 - Eugénie Garagnon (Rennes)
 - Violette Jégou (Rennes)
 - Vincent Ferrari (Ulm)
 - Paul-Swann Puel (Lyon)
 - Cyann Bernard (Ulm)
 - Mina Horch (Paris Saclay)
 - Quentin Zanini (Rennes)
 - Maël Chantreau (Lyon)
 - Antoine Velut (Lyon)
 - Florian Taraveau (Lyon)
 - Arthur DUMAS (Rennes)
 - Larg Gruson (Ulm)
 - Vincent Badetti (Rennes)
 - Iwan Quémerais (Lyon)
 - Iris Magniez--Papillon (Lyon)
 - Corentin Debruille (Lyon)
 - Sylvain Gay (Ulm)
 - Pierre-Marie Desbouchages (Rennes)
 - Adrien Fauste-Gay (Ulm)
[GN] Le Dilemme de Chronos : 4 personnes
 - Valentin Richard (Paris Saclay)
 - petit gastéropode (Paris Saclay)
 - Nicolas Fabiano (Ulm)
 - Adoracion Guzman Garcia (Ulm)
[

In [228]:
cast_perso = {}
for (act, pl) in matchings.items():
    for p in pl:
        name = participants[p]
        if name in cast_perso:
            cast_perso[name] += 1
        else:
            cast_perso[name] = 1

multiples = [p for (p, v) in cast_perso.items() if v > 1]
for p in multiples:
    print(p)
    for (act, pl) in matchings.items():
        if p in [participants[a] for a in pl]:
            print(" - ", activites[act])

sans_cast = [p for p in participants.values() if p not in cast_perso.keys()]
print(len(sans_cast))
sans_cast


Antoine Guilmin-Crépon (Paris Saclay)
 -  [GN] L'engrenage infernal
 -  [Jeux] Fresque du Climat - Dimanche Matin
Mina Horch (Paris Saclay)
 -  [GN] L'engrenage infernal
 -  [GN] Documents Secrets
Quentin Zanini (Rennes)
 -  [GN] L'engrenage infernal
 -  [JdR] Demi-Dieu - Dimanche
Maël Chantreau (Lyon)
 -  [GN] L'engrenage infernal
 -  [JdR] [d100 béta] Les larmes de Melvumia
Antoine Velut (Lyon)
 -  [GN] L'engrenage infernal
 -  [GN] Abyme Secret
Florian Taraveau (Lyon)
 -  [GN] L'engrenage infernal
 -  [GN] L'Ecosysteme
Arthur DUMAS (Rennes)
 -  [GN] L'engrenage infernal
 -  [JdR] Préservez la Flamme
Larg Gruson (Ulm)
 -  [GN] L'engrenage infernal
 -  [GN] Abyme Secret
Vincent Badetti (Rennes)
 -  [GN] L'engrenage infernal
 -  [GN] Black Pram
Iwan Quémerais (Lyon)
 -  [GN] L'engrenage infernal
 -  [JdR] Larmes & Poussières
Iris Magniez--Papillon (Lyon)
 -  [GN] L'engrenage infernal
 -  [JdR] Préservez la Flamme
Corentin Debruille (Lyon)
 -  [GN] L'engrenage infernal
 -  [GN] Abyme Se

['Alexandre Marthe (Lyon)',
 'gael gallot (Lyon)',
 'Danaé Guiserix (Lyon)',
 'Alexis MARISSAËL (Rennes)',
 'Emma Sabre (Paris Saclay)',
 'Adrienne Le Meur (Rennes)',
 'Pierre Dedieu (Lyon)',
 'Théo Décamps (Ulm)',
 'Sadek Al-Jibouri (Lyon)',
 'Erwann LOULERGUE (Paris Saclay)',
 'Léo Thiébaud (Lyon)',
 'Théo Rudkiewicz (Paris Saclay)',
 'Louis Vassaux (Ulm)']

In [221]:
matchings

{27: [1,
  68,
  2,
  20,
  126,
  62,
  109,
  6,
  230,
  21,
  237,
  91,
  238,
  177,
  231,
  96,
  227,
  41,
  39,
  101,
  202,
  138,
  144,
  51,
  145,
  43,
  137,
  187,
  148],
 5: [35, 69, 80, 5],
 37: [168, 221, 132, 115, 215, 156, 120, 211, 17, 48, 11, 233],
 18: [197, 74, 33, 9, 14, 128, 30],
 21: [208, 223, 100, 98, 174, 83, 157, 81, 147, 117],
 8: [239, 49, 99, 123, 59],
 11: [175, 73, 87, 28, 71, 191, 25, 118],
 15: [63, 194, 50, 12, 161, 206, 112, 204, 181, 135, 189, 235, 94],
 19: [153, 163, 178, 67, 217, 96, 85],
 16: [162, 88, 108, 51],
 9: [165,
  155,
  166,
  173,
  151,
  58,
  64,
  226,
  148,
  184,
  37,
  60,
  189,
  135,
  75,
  143,
  186,
  101,
  160,
  92],
 26: [8, 113, 158, 188, 139, 195, 218, 116, 86, 176, 107, 226],
 17: [105, 29, 228, 90, 225, 94, 204, 224, 214, 84, 180, 15, 24],
 14: [159, 42, 185, 236, 241],
 24: [140, 76, 110, 23],
 10: [31, 219, 137, 152, 187, 154, 229, 89],
 2: [181, 170, 136, 41],
 3: [196, 39, 43, 107, 138, 44],
 6: 

In [219]:
conflits

{2: [3, 5, 6, 24, 25],
 3: [2, 5, 6, 24, 25],
 5: [2, 3, 6, 24, 25],
 6: [2, 3, 5, 24, 25],
 8: [11, 17, 18, 26, 27, 28],
 9: [13, 16],
 10: [13, 16],
 11: [8, 17, 18, 26, 27, 28],
 13: [9, 10, 16],
 14: [15, 18, 26, 27],
 15: [14, 18, 26, 27, 29, 37],
 16: [9, 10, 13],
 17: [8, 11, 18, 26, 27],
 18: [8, 11, 14, 15, 17, 26, 27, 29, 37],
 19: [20, 21, 23, 31],
 20: [19, 21, 23, 31],
 21: [19, 20, 23, 31],
 23: [19, 20, 21, 31],
 24: [2, 3, 5, 6],
 25: [2, 3, 5, 6],
 26: [8, 11, 14, 15, 17, 18, 27, 28, 29, 37],
 27: [8, 11, 14, 15, 17, 18, 26, 29, 37],
 28: [8, 11, 26],
 29: [15, 18, 26, 27, 37],
 31: [19, 20, 21, 23],
 37: [15, 18, 26, 27, 29]}